In [1]:
import pandas as pd
import numpy as np
import time
import os

import spacy

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import kmodes
from kmodes.kprototypes import KPrototypes

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv1D, MaxPool1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer

In [84]:
data_folder = "../Data-Analytics-DRMIS-SAP-Production-DATA/"

df = pd.read_csv(data_folder + 'LAV 6.csv', encoding = 'ISO-8859-1', index_col = 1).reset_index(drop = False)
#re-download df with MaintActivityType
df = df[df['MaintActivityType']=='L01'].reset_index(drop = True)

In [50]:
list(df.columns)

['Order Type',
 'Order',
 'Created on',
 'Basic start date',
 'Description',
 'Total planned costs',
 'PM Orders planned hours',
 'PM Orders actual hours',
 'Tail Number / Identifier',
 'User Status',
 'Main work center',
 'Vehicle Type',
 'Work Type',
 'Work Type Text',
 'Scheduled Fin. Time',
 'Scheduled finish',
 'Scheduled start',
 'Priority',
 'Priority text',
 'Priority type',
 'PM order',
 'Maintenance item',
 'MaintActivityType',
 'MaintActivityType.1',
 'Level',
 'Level of work',
 'License plate number',
 'Actual Finish Time',
 'Actual Order Finish Date',
 'Actual release date',
 'Actual start date',
 'ActualStartTime',
 'Available from Date',
 'Available from time',
 'Available to date',
 'Available to time',
 'Basic finish date',
 'Estimated costs',
 'Flow through costs',
 'Maintenance strategy',
 '1 encoded word',
 '2 encoded word',
 '3 encoded word',
 '4 encoded word',
 '5 encoded word',
 '6 encoded word',
 '7 encoded word',
 '8 encoded word',
 '9 encoded word',
 '10 encod

In [86]:
df['Main work center'].value_counts()[0:50]

FW2B---1    582
FC1L-1V1    335
FW2LA-V1    295
FC2L-4V1    280
FQ2LA-V1    225
CLLL-2V2    202
FQHLA-V1    193
FW1B---1    187
FQ2M--W1    181
FABB---1    179
FW1LA2V1    152
FQ2R--E1    145
FQ1L-1V1    145
FC2R--E1    136
FW1LA3V1    135
FC2M--W1    134
CLLK-3-2     99
FW2R---1     89
FW1LA1V1     87
FQ4LA-V1     85
FQ1R--E1     85
FC1R--E1     80
FABLA3V1     80
FQHR--E1     75
FABRA1E1     73
XW1B---1     71
FQ6LA-V1     69
FABMA-W1     66
FQ6M--W1     61
FW1R--E1     57
FQBHP-V2     55
FW1M--W1     55
FC2L-2V1     54
FQ1Q--L1     53
XC1B---1     51
XQ2B---1     50
FW2M---1     48
FWBLA1V2     48
LWC-1850     43
LWC-1847     41
FW2Q--L1     40
FC1A---1     40
LWC-1849     39
FW1Q--L1     38
CLLLA-V2     37
XC2B---1     36
FC1M--W1     36
FC1Q--L1     36
FW6B---1     29
FQ1M--W1     29
Name: Main work center, dtype: int64

In [87]:
len(df['Main work center'].value_counts())

239

In [70]:
df[df['Main work center'] == 'FW2R---1']

,Order Type,Order,Created on,Basic start date,Description,Total planned costs,PM Orders planned hours,PM Orders actual hours,Tail Number / Identifier,User Status,...,3 encoded word,4 encoded word,5 encoded word,6 encoded word,7 encoded word,8 encoded word,9 encoded word,10 encoded word,11 encoded word,12 encoded word
1297,L001,1102484300,2019-08-22,2019-08-22,EO - 03422 - Tango box,109.65,1.0,1.0,2011-03422,OAPP,...,617,486,0,0,0,0,0,0,0,0
1298,L001,1102557876,2020-01-28,2020-01-28,EO - 03427 - Repair,"49,852.08",3.0,3.0,2011-03427,OAPP,...,8,0,0,0,0,0,0,0,0,0
1299,L001,1102220632,2018-01-17,2018-01-17,EO - 03427 - Insp.Rep. - Blackout Cover,"1,531.00",1.5,1.5,2011-03427,OAPP,...,5,67,1447,451,0,0,0,0,0,0
1300,L001,1102606581,2020-06-30,2020-06-30,EO - 03427 - Brightness Knob Gnr Sight,125.42,1.0,0.0,2011-03427,TAPP AMAT,...,597,490,654,414,0,0,0,0,0,0
1301,L001,1102380750,2019-01-07,2018-12-03,EO - 03433 - DVE,"20,314.66",3.5,1.5,2011-03433,OAPP,...,112,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1405,L001,1102357599,2018-11-01,2018-10-31,EO - 03581 - IRTAS,"99,643.12",22.5,22.5,2011-03581,OAPP,...,101,0,0,0,0,0,0,0,0,0
1406,L001,1102160060,2017-09-01,2017-09-01,EO - 03612 Annual Insp,261.43,2.5,2.5,2011-03612,OAPP,...,41,5,0,0,0,0,0,0,0,0
1407,L001,1102052050,2017-01-30,2017-01-30,EO-03612-REMOVE CSS-002 SIGHTS,627.42,6.0,6.0,2011-03612,OAPP,...,667,78,698,668,0,0,0,0,0,0
1408,L001,1102250352,2018-03-13,2018-03-13,EO - 03612 - Insp.Rep. - Code 57,"63,027.45",3.0,3.0,2011-03612,OAPP,...,5,67,232,1252,0,0,0,0,0,0


In [88]:
text = df['Description'].astype(str).tolist()
token = Tokenizer()
token.fit_on_texts(text)
vocab = len(token.index_word) + 1
encoded_text = token.texts_to_sequences(text)

max_length = max([len(x) for x in encoded_text])

token_dict = token.__dict__
word_list1 = list(token_dict['index_word'].values())
drop_list = []

In [89]:

for i in range(1,max_length+1):
    df[str(i)+ ' encoded word'] = [0]*len(df)
    for n in range(len(df)):
        try:
            df[str(i)+ ' encoded word'][n] = encoded_text[n][i-1]
        except:
            pass

c:\users\olfoa\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [90]:
encoded_words_lists =[]

for i in range(1,13):
    encoded_words_lists.append(str(i)+ ' encoded word')
ready_df = df[['Order','Description','Total planned costs', 'Main work center','PM Orders planned hours']+encoded_words_lists]

In [94]:
input_df = ready_df.drop(['Order', 'Description', 'Main work center'], axis = 1)
for i in range(len(input_df)):
    input_df['Total planned costs'][i] =float(input_df['Total planned costs'][i].replace(',',''))

c:\users\olfoa\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [95]:
input_df.head()

,Total planned costs,PM Orders planned hours,1 encoded word,2 encoded word,3 encoded word,4 encoded word,5 encoded word,6 encoded word,7 encoded word,8 encoded word,9 encoded word,10 encoded word,11 encoded word,12 encoded word
0,172711,20.0,14,75,405,1,2,61,301,19,18,0,0,0
1,8214.99,8.0,14,75,215,1,2,133,382,55,0,0,0,0
2,9469.83,20.0,14,75,202,1,2,750,133,55,19,18,0,0
3,173727,19.0,75,11,93,25,1154,0,0,0,0,0,0,0
4,9356.58,15.0,75,11,136,254,183,88,0,0,0,0,0,0


In [96]:
input_array = input_df.values
input_array[:, 0] = input_array[0:, 0].astype(float)
input_array[:, 1] = input_array[0:, 1].astype(float)

for i in range(2,14):
    input_array[:, i] = input_array[0:, i].astype(str)


In [99]:
categorical_data = []
for i in range(2,14):
    categorical_data.append(i)
    
kproto = KPrototypes(n_clusters = 4, verbose = 2, max_iter = 20)
clusters = kproto.fit_predict(input_array, categorical = categorical_data)

Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 1, iteration: 1/20, moves: 683, ncost: 593451694222.5553
Run: 1, iteration: 2/20, moves: 218, ncost: 436080206140.27997
Run: 1, iteration: 3/20, moves: 89, ncost: 406804651724.97504
Run: 1, iteration: 4/20, moves: 88, ncost: 400305352772.4085
Run: 1, iteration: 5/20, moves: 45, ncost: 399371589322.9282
Run: 1, iteration: 6/20, moves: 45, ncost: 397398510424.30115
Run: 1, iteration: 7/20, moves: 46, ncost: 394753198319.9525
Run: 1, iteration: 8/20, moves: 35, ncost: 393887788714.6567
Run: 1, iteration: 9/20, moves: 13, ncost: 393797839908.1039
Run: 1, iteration: 10/20, moves: 6, ncost: 393773314204.4634
Run: 1, iteration: 1

In [103]:
ready_df['cluster'] = list(clusters)

c:\users\olfoa\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [110]:
ready_df[ready_df['cluster']==0]['Main work center'].value_counts()[0:10]

LWC-1847    17
FC2R--E1    11
FQ1R--E1     6
FW2R---1     6
FQ2R--E1     4
FQHLA-V1     4
FW2B---1     3
XWBB---2     3
FQHR--E1     3
FW1B---1     3
Name: Main work center, dtype: int64

In [111]:
ready_df[ready_df['cluster']==1]['Main work center'].value_counts()[0:10]

FABB---1    44
FC2L-4V1    42
FW2B---1    35
FW2LA-V1    31
FW2R---1    30
FC2R--E1    25
FQ2R--E1    22
FABLA3V1    21
FQ1R--E1    19
FC1R--E1    15
Name: Main work center, dtype: int64

In [112]:
ready_df[ready_df['cluster']==2]['Main work center'].value_counts()[0:10]

FW2B---1    532
FC1L-1V1    328
FW2LA-V1    260
FC2L-4V1    235
FQ2LA-V1    210
CLLL-2V2    192
FQHLA-V1    176
FW1B---1    164
FW1LA2V1    141
FQ2M--W1    141
Name: Main work center, dtype: int64

In [113]:
ready_df[ready_df['cluster']==3]['Main work center'].value_counts()[0:10]

FQ2M--W1    34
FW2R---1    14
FW2B---1    12
FQ2R--E1    12
FC1R--E1    11
FC2R--E1    11
FABRA1E1     9
FQ1R--E1     8
FA6B---1     7
FW1B---1     6
Name: Main work center, dtype: int64

In [117]:
len(ready_df[ready_df['cluster']==0])

90

array(['XWBB---2', 'XW7B---1', 'XW6B---1', 'XW4B---1', 'XW2B---1',
       'XW1B---1', 'XQBB---2', 'XQBB---1', 'XQ6B---1', 'XQ4B---1',
       'XQ2B---1', 'XQ1B---1', 'XCBB---2', 'XC4B---1', 'XC2B---1',
       'XC1B---1', 'TOR-WRW1', 'TOR-WKP1', 'TOR-WEL1', 'LWC-7753',
       'LWC-7752', 'LWC-6923', 'LWC-6847', 'LWC-6422', 'LWC-5246',
       'LWC-3543', 'LWC-3447', 'LWC-2112', 'LWC-1853', 'LWC-1852',
       'LWC-1850', 'LWC-1849', 'LWC-1848', 'LWC-1847', 'LWC-1779',
       'LWC-1775', 'LWC-1774', 'LWC-1764', 'LWC-1754', 'LWC-1702',
       'LWC-1701', 'LWC-1430', 'LWC-0870', 'LWC-0106', 'LWC-0105',
       'GW7O--M1', 'GW7L--V1', 'GQ7S--V1', 'GQ7LB-V1', 'GCBL--V1',
       'GC7O--M1', 'GC7LB-V1', 'GC7B---1', 'G-BR--E2', 'G-BLA-V2',
       'G-BK-5-2', 'G-BK-1-2', 'G-BI---2', 'G-BB---2', 'G-BA---2',
       'FWKLB1V1', 'FWKG----', 'FWKB---2', 'FWJT---2', 'FWJO--M2',
       'FWJM--W2', 'FWJL---2', 'FWBT---2', 'FWBS--V2', 'FWBR-2E2',
       'FWBOK-M2', 'FWBO--M2', 'FWBO--M1', 'FWBM--W2', 'FWBLB1

In [ ]:
simalirity_comparison_words = ['transmission', 'engine', 'brake', 'suspension', 'axle', 'turret', 'gun', 'sensor', 'sight', 'controls', 'hull']

In [94]:
token.__dict__.keys()

dict_keys(['word_counts', 'word_docs', 'filters', 'split', 'lower', 'num_words', 'document_count', 'char_level', 'oov_token', 'index_docs', 'word_index', 'index_word'])

In [ ]:
token_dict = token.__dict__
word_list1 = list(token_dict['index_word'].values())
drop_list = []

remove:
- non words (codes, ect)
- numbers
- stop words

In [13]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import spacy
nlp = spacy.load('en_vectors_web_lg')
def similarityy(x,y):
    return(nlp(x).similarity(nlp(y)))


In [ ]:
#remove numbers
for i in range(len(word_list1)):
    try:
        if type(float(word_list1[i])) == type(1.0):
            word_list1.pop(i)
    
    except:
        pass
    
for i in range(len(word_list1)):
    try:
        if type(float(word_list1[i])) == type(1):
            word_list1.pop(i)
    
    except:
        pass

for i in range(len(word_list1)):
    try:
        if len(word_list1[i]) <= 2:
            word_list1.pop(i)
    
    except:
        pass


#remove non words
for i in range(len(word_list1)):
    try:
        if similarityy(word_list1[i], 'vehicle') < 0.15:
            word_list1.pop(i)
    except:
        pass
for i in range(len(word_list1)):
    try:
        if type(float(word_list1[i])) == type(1.0):
            word_list1.pop(i)
    
    except:
        pass
    
for i in range(len(word_list1)):
    try:
        if type(float(word_list1[i])) == type(1):
            word_list1.pop(i)
    
    except:
        pass

for i in range(len(word_list1)):
    try:
        if len(word_list1[i]) <= 2:
            word_list1.pop(i)
    
    except:
        pass
    
word_counts = dict(token.__dict__['word_counts'])
real_words = {}
for word in word_list1:
    real_words[word] = word_counts[word]

In [110]:
len(real_words)

991

In [85]:
df['Description'][0]

'A-2VP-03403-LAV 6- POWER PACK N/S'

,Order Type,Created on,Basic start date,Description,Total planned costs,PM Orders planned hours,PM Orders actual hours,Tail Number / Identifier,User Status,Main work center,...,Actual start date,ActualStartTime,Available from Date,Available from time,Available to date,Available to time,Basic finish date,Estimated costs,Flow through costs,Maintenance strategy
0,L001,2016-04-24,2016-04-24,A-2VP-03403-LAV 6- POWER PACK N/S,"172,711.02",20.0,55.0,2011-03403,OAPP HPR1,XWBB---2,...,2016-04-25,12:00:00 AM,NaN,12:00:00 AM,NaN,12:00:00 AM,2016-04-24,0,NaN,NaN
1,L001,2016-04-12,2016-04-12,A-2VP-03427-LAV 6-Front Left Tire,"8,214.99",8.0,8.0,2011-03427,OAPP,XWBB---2,...,2016-04-14,12:00:00 AM,NaN,12:00:00 AM,NaN,12:00:00 AM,2016-04-12,0,NaN,NaN
2,L001,2016-04-12,2016-04-12,A-2VP-03452-LAV 6-Hydrop/Front tire N/S,"9,469.83",20.0,10.0,2011-03452,OAPP,XWBB---2,...,2016-04-18,12:00:00 AM,NaN,12:00:00 AM,NaN,12:00:00 AM,2016-04-12,0,NaN,NaN
3,L001,2019-04-10,2019-04-10,2VP - LAV6 - 03409 - Replace PP,"173,727.17",19.0,19.0,2011-03409,OAPP,XWBB---2,...,2019-04-10,12:00:00 AM,NaN,12:00:00 AM,NaN,12:00:00 AM,2019-04-10,0,NaN,NaN
4,L001,2019-04-08,2019-04-08,2VP - LAV6 - 03411 - L1 STRUT NS,"9,356.58",15.0,12.0,2011-03411,OAPP HPR1,XWBB---2,...,2019-04-08,12:00:00 AM,NaN,12:00:00 AM,NaN,12:00:00 AM,2019-04-08,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13666,L001,2016-05-27,2016-08-23,LAV 6.0 INSPECTION REPARATION - 03381,0,0.0,0.0,2011-03381,OAPP,202PROD,...,NaN,12:00:00 AM,NaN,12:00:00 AM,NaN,12:00:00 AM,2017-05-01,0,NaN,NaN
13667,L001,2016-05-27,2016-05-27,LAV 6.0 INSPECTION REPARATION - 03383,0,0.0,0.0,2011-03383,OAPP,202PROD,...,NaN,12:00:00 AM,NaN,12:00:00 AM,NaN,12:00:00 AM,2016-05-27,0,NaN,NaN
13668,L001,2017-02-06,2017-02-06,LAV 6.0 Engine ECM Flash,0,0.0,0.0,2011-03492,OAPP,202PROD,...,NaN,12:00:00 AM,NaN,12:00:00 AM,NaN,12:00:00 AM,2017-02-06,0,NaN,NaN
13669,L001,2018-02-26,2018-07-10,INSTALLATION BLINDAGE,"198,383.79",0.0,0.0,2016-03868,OAPP,202PROD,...,NaN,12:00:00 AM,NaN,12:00:00 AM,NaN,12:00:00 AM,2018-10-19,0,NaN,NaN


In [ ]:
#clustering should combine features from description, as well as PM orders planned hours, Total planned costs, order type

In [16]:
print(help(token))

Help on Tokenizer in module keras_preprocessing.text object:

class Tokenizer(builtins.object)
 |  Text tokenization utility class.
 |  
 |  This class allows to vectorize a text corpus, by turning each
 |  text into either a sequence of integers (each integer being the index
 |  of a token in a dictionary) or into a vector where the coefficient
 |  for each token could be binary, based on word count, based on tf-idf...
 |  
 |  # Arguments
 |      num_words: the maximum number of words to keep, based
 |          on word frequency. Only the most common `num_words-1` words will
 |          be kept.
 |      filters: a string where each element is a character that will be
 |          filtered from the texts. The default is all punctuation, plus
 |          tabs and line breaks, minus the `'` character.
 |      lower: boolean. Whether to convert the texts to lowercase.
 |      split: str. Separator for word splitting.
 |      char_level: if True, every character will be treated as a token.

In [ ]:
#create ranges of days WIP